# Tool Calling & Agent

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint
import json

`(3) Langsmith tracing 설정`

In [3]:
# Langsmith tracing 여부를 확인 (true: langsmith 추척 활성화, false: langsmith 추척 비활성화)
import os
print(os.getenv('LANGSMITH_TRACING'))

true


---

## Tool Calling 개념

- **Tool Calling**은 LLM이 외부 시스템과 상호작용하기 위한 핵심 메커니즘
- **구조화된 출력**: LLM이 정의된 스키마에 따라 함수 호출 정보를 생성
- **외부 시스템 연동**: API, 데이터베이스, 파일 시스템 등과 연결
- **자동 검증**: 스키마 기반으로 입력 파라미터 자동 검증
- **유연한 확장**: 새로운 도구를 쉽게 추가하고 제거 가능


![Tool Calling Concept](https://python.langchain.com/assets/images/tool_calling_concept-552a73031228ff9144c7d59f26dedbbf.png)


[참조] https://python.langchain.com/docs/concepts/tool_calling/

---

### 1. 기본적인 Tool 생성

#### 1.1 @tool 데코레이터 사용법

- **@tool 데코레이터**로 함수에 스키마 정보 추가

- **함수와 스키마** 간 자동 연결로 도구 생성

In [4]:
from langchain_core.tools import tool
from typing import Literal

@tool
def add(a: int, b: int) -> int:
    """두 정수를 더합니다.
    
    Args:
        a: 첫 번째 정수
        b: 두 번째 정수
    
    Returns:
        두 수의 합
    """
    return a + b

@tool
def multiply(a: int, b: int) -> int:
    """두 정수를 곱합니다.
    
    Args:
        a: 첫 번째 정수
        b: 두 번째 정수
    
    Returns:
        두 수의 곱
    """
    return a * b

# 도구 실행 테스트
print(add.invoke({'a': 3, 'b': 5}))  # 8
print(multiply.invoke({'a': 4, 'b': 6}))  # 24

8
24


#### 1.2 복잡한 Tool 예제

In [5]:
from datetime import datetime
from typing import Optional

@tool
def get_current_time(format_type: Literal["date", "time", "both"] = "both") -> str:
    """현재 날짜와 시간을 반환합니다.
    
    Args:
        format_type: 반환할 형식 ('date', 'time', 'both' 중 선택)
    
    Returns:
        포맷된 날짜/시간 문자열
    """
    now = datetime.now()
    
    if format_type == "date":
        return now.strftime("%Y년 %m월 %d일")
    elif format_type == "time":
        return now.strftime("%H시 %M분 %S초")
    else:
        return now.strftime("%Y년 %m월 %d일 %H시 %M분 %S초")

@tool
def calculate_age(birth_year: int) -> str:
    """태어난 년도를 입력받아 나이를 계산합니다.
    
    Args:
        birth_year: 태어난 년도 (예: 1990)
    
    Returns:
        계산된 나이
    """
    current_year = datetime.now().year
    age = current_year - birth_year
    return f"{age}세"

### 2. Tool을 LLM에 연결하기

#### 2.1 기본 연결 방법


In [6]:
from langchain_openai import ChatOpenAI

# LLM 모델 초기화
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# 도구 목록 생성
tools = [add, multiply, get_current_time, calculate_age]

# 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools)

# 계산 도구 호출 테스트
response = llm_with_tools.invoke("15와 23을 더해주세요")
print(response)

content='' additional_kwargs={'tool_calls': [{'id': 'call_OVWFVw09ljA4O1Xz2znxH7WL', 'function': {'arguments': '{"a":15,"b":23}', 'name': 'add'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 249, 'total_tokens': 266, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-C26WBQdEvAYxVsqGagdBO35Smfj2C', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--f8e86967-db42-44c8-8f25-bda72d44af61-0' tool_calls=[{'name': 'add', 'args': {'a': 15, 'b': 23}, 'id': 'call_OVWFVw09ljA4O1Xz2znxH7WL', 'type': 'tool_call'}] usage_metadata={'input_tokens': 249, 'output_tokens': 17, 'total_tokens': 266, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'ou

In [7]:
print(response.tool_calls)

[{'name': 'add', 'args': {'a': 15, 'b': 23}, 'id': 'call_OVWFVw09ljA4O1Xz2znxH7WL', 'type': 'tool_call'}]


In [8]:
# 날짜 시간 도구 호출 테스트
response = llm_with_tools.invoke("현재 시간을 알려주세요")
print(response) 

content='' additional_kwargs={'tool_calls': [{'id': 'call_1j9ImJDtiqHcITZGTV3nAHWZ', 'function': {'arguments': '{"format_type":"both"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 246, 'total_tokens': 262, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-C26WfnDWI4eD0jg6hJq5s3lKfg6ut', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--fe8d2ef9-fd57-4e66-a46a-c9b056707a21-0' tool_calls=[{'name': 'get_current_time', 'args': {'format_type': 'both'}, 'id': 'call_1j9ImJDtiqHcITZGTV3nAHWZ', 'type': 'tool_call'}] usage_metadata={'input_tokens': 246, 'output_tokens': 16, 'total_tokens': 262, 'input_token_detail

In [9]:
print(response.tool_calls)

[{'name': 'get_current_time', 'args': {'format_type': 'both'}, 'id': 'call_1j9ImJDtiqHcITZGTV3nAHWZ', 'type': 'tool_call'}]


In [10]:
# 나이 계산 도구 호출 테스트
response = llm_with_tools.invoke("내가 태어난 년도는 1990년입니다. 내 나이를 알려주세요.")
print(response)

content='' additional_kwargs={'tool_calls': [{'id': 'call_zSvxP6xgRpWswBANvgNvWwJ1', 'function': {'arguments': '{"birth_year":1990}', 'name': 'calculate_age'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 261, 'total_tokens': 277, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-C26X1vNpZlsuoNfowBA9dDyhSfrF5', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--1f96ca25-3025-44c8-ad2d-14049d4a4f5e-0' tool_calls=[{'name': 'calculate_age', 'args': {'birth_year': 1990}, 'id': 'call_zSvxP6xgRpWswBANvgNvWwJ1', 'type': 'tool_call'}] usage_metadata={'input_tokens': 261, 'output_tokens': 16, 'total_tokens': 277, 'input_token_details': {'audio'

In [11]:
print(response.tool_calls)

[{'name': 'calculate_age', 'args': {'birth_year': 1990}, 'id': 'call_zSvxP6xgRpWswBANvgNvWwJ1', 'type': 'tool_call'}]


#### 2.2 Tool Choice 옵션


In [12]:
# 특정 도구를 강제로 사용하도록 설정
llm_forced_add = llm.bind_tools(tools, tool_choice="add")
response = llm_forced_add.invoke("10+20계산을 해주세요")
print(response)

content='' additional_kwargs={'tool_calls': [{'id': 'call_KDOVW2Pa25VvBraH5J4PwjOe', 'function': {'arguments': '{"a":10,"b":20}', 'name': 'add'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 258, 'total_tokens': 267, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-C26YCMgK8YUkJVN229FMwi5JW3EXx', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--fa67ef18-7a42-4a7d-aa4e-3271d279a699-0' tool_calls=[{'name': 'add', 'args': {'a': 10, 'b': 20}, 'id': 'call_KDOVW2Pa25VvBraH5J4PwjOe', 'type': 'tool_call'}] usage_metadata={'input_tokens': 258, 'output_tokens': 9, 'total_tokens': 267, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_tok

In [13]:
print(response.tool_calls)

[{'name': 'add', 'args': {'a': 10, 'b': 20}, 'id': 'call_KDOVW2Pa25VvBraH5J4PwjOe', 'type': 'tool_call'}]


In [14]:
# 반드시 어떤 도구든 사용하도록 설정
llm_must_use_tool = llm.bind_tools(tools, tool_choice="any")
response = llm_must_use_tool.invoke("안녕하세요")
print(response)

content='' additional_kwargs={'tool_calls': [{'id': 'call_yycg8TlIiWb4Jf0daL4jydT4', 'function': {'arguments': '{"format_type":"both"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 244, 'total_tokens': 260, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-C26YNPwviLCAT4lQb8WNjOPz1WPS9', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--9efe953d-7782-439a-93de-762345d6bba5-0' tool_calls=[{'name': 'get_current_time', 'args': {'format_type': 'both'}, 'id': 'call_yycg8TlIiWb4Jf0daL4jydT4', 'type': 'tool_call'}] usage_metadata={'input_tokens': 244, 'output_tokens': 16, 'total_tokens': 260, 'input_token_detail

In [15]:
print(response.tool_calls)

[{'name': 'get_current_time', 'args': {'format_type': 'both'}, 'id': 'call_yycg8TlIiWb4Jf0daL4jydT4', 'type': 'tool_call'}]


In [16]:
# 병렬 도구 호출 비활성화
llm_sequential = llm.bind_tools(tools, parallel_tool_calls=False)
response = llm_sequential.invoke("3+5와 4*6을 계산해주세요")
print(response)

content='' additional_kwargs={'tool_calls': [{'id': 'call_9kW3b2Lo8IgccZyZWudDsbTB', 'function': {'arguments': '{"a":3,"b":5}', 'name': 'add'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 253, 'total_tokens': 270, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-C26Yn3IlhKgn3lO845VjcyXXUBF0o', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--ae1e1d93-71f5-4af2-ac98-12efe6632def-0' tool_calls=[{'name': 'add', 'args': {'a': 3, 'b': 5}, 'id': 'call_9kW3b2Lo8IgccZyZWudDsbTB', 'type': 'tool_call'}] usage_metadata={'input_tokens': 253, 'output_tokens': 17, 'total_tokens': 270, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output

In [17]:
print(response.tool_calls)

[{'name': 'add', 'args': {'a': 3, 'b': 5}, 'id': 'call_9kW3b2Lo8IgccZyZWudDsbTB', 'type': 'tool_call'}]


In [18]:
# 병렬 도구 호출 활성화
llm_sequential = llm.bind_tools(tools, parallel_tool_calls=True)
response = llm_sequential.invoke("3+5와 4*6을 계산해주세요")
print(response)

content='' additional_kwargs={'tool_calls': [{'id': 'call_8U4Wi348dqABztgIi6e7AbYg', 'function': {'arguments': '{"a": 3, "b": 5}', 'name': 'add'}, 'type': 'function'}, {'id': 'call_o47Pcy12lNz9c98qyAf2yflx', 'function': {'arguments': '{"a": 4, "b": 6}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 253, 'total_tokens': 303, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-C26Z6EnJupzzMoO0LTwMXjIvkk4oA', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--b330adde-5840-49ce-9537-6cba49007954-0' tool_calls=[{'name': 'add', 'args': {'a': 3, 'b': 5}, 'id': 'call_8U4Wi348dqABztgIi6e7AbYg', 'type': 'tool_call'}, {'name': 

In [19]:
print(response.tool_calls)

[{'name': 'add', 'args': {'a': 3, 'b': 5}, 'id': 'call_8U4Wi348dqABztgIi6e7AbYg', 'type': 'tool_call'}, {'name': 'multiply', 'args': {'a': 4, 'b': 6}, 'id': 'call_o47Pcy12lNz9c98qyAf2yflx', 'type': 'tool_call'}]


In [20]:
response

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8U4Wi348dqABztgIi6e7AbYg', 'function': {'arguments': '{"a": 3, "b": 5}', 'name': 'add'}, 'type': 'function'}, {'id': 'call_o47Pcy12lNz9c98qyAf2yflx', 'function': {'arguments': '{"a": 4, "b": 6}', 'name': 'multiply'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 253, 'total_tokens': 303, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-C26Z6EnJupzzMoO0LTwMXjIvkk4oA', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--b330adde-5840-49ce-9537-6cba49007954-0', tool_calls=[{'name': 'add', 'args': {'a': 3, 'b': 5}, 'id': 'call_8U4Wi348dqABztgIi6e7AbYg', 'type': 'tool_cal

### 3. Tool Calling 처리하기

#### 3.1 Tool Calls 확인하고 실행하기

In [21]:
def execute_tool_calls(message, tools_dict):
    """Tool calls를 실행하고 결과를 반환합니다."""
    results = []
    
    for tool_call in message.tool_calls:
        tool_name = tool_call['name']
        tool_args = tool_call['args']
        
        if tool_name in tools_dict:
            tool = tools_dict[tool_name]
            result = tool.invoke(tool_args)
            results.append({
                'tool_call_id': tool_call['id'],
                'tool_name': tool_name,
                'result': result
            })
    
    return results

# 도구 딕셔너리 생성
tools_dict = {tool.name: tool for tool in tools}

# 테스트
response = llm_with_tools.invoke("오늘 날짜를 알려주세요")
if response.tool_calls:
    results = execute_tool_calls(response, tools_dict)
    for result in results:
        print(f"도구: {result['tool_name']}")
        print(f"결과: {result['result']}")

도구: get_current_time
결과: 2025년 08월 08일


#### 3.2 완전한 대화 흐름

In [22]:
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

def chat_with_tools(user_input: str, llm_with_tools, tools_dict):
    """도구를 사용한 완전한 대화 처리"""
    messages = [HumanMessage(content=user_input)]
    
    # LLM 호출
    ai_response = llm_with_tools.invoke(messages)
    messages.append(ai_response)
    
    # Tool calls가 있으면 실행
    if ai_response.tool_calls:
        for tool_call in ai_response.tool_calls:
            tool_name = tool_call['name']
            tool_args = tool_call['args']
            tool_id = tool_call['id']
            
            # 도구 실행
            if tool_name in tools_dict:
                tool_result = tools_dict[tool_name].invoke(tool_args)
                # ToolMessage 추가
                messages.append(
                    ToolMessage(
                        content=str(tool_result),
                        tool_call_id=tool_id
                    )
                )
        
        # 최종 응답 생성
        final_response = llm_with_tools.invoke(messages)
        return final_response.content
    else:
        return ai_response.content

# 테스트
result = chat_with_tools("15 곱하기 8은 얼마인가요?", llm_with_tools, tools_dict)
print(result)

15 곱하기 8은 120입니다.


### 4. 고급 Tool Calling 기법

#### 4.1 오류 처리가 포함된 Tool


In [23]:
@tool
def safe_divide(a: float, b: float) -> str:
    """두 수를 나눕니다. (0으로 나누기 방지)
    
    Args:
        a: 피제수
        b: 제수
    
    Returns:
        나눗셈 결과 또는 오류 메시지
    """
    try:
        if b == 0:
            return "오류: 0으로 나눌 수 없습니다."
        result = a / b
        return f"{a} ÷ {b} = {result}"
    except Exception as e:
        return f"계산 오류: {str(e)}"

@tool
def validate_input(value: str, data_type: Literal["int", "float", "email"]) -> str:
    """입력값을 검증합니다.
    
    Args:
        value: 검증할 값
        data_type: 데이터 타입 ('int', 'float', 'email')
    
    Returns:
        검증 결과
    """
    import re
    
    if data_type == "int":
        try:
            int(value)
            return f"'{value}'는 유효한 정수입니다."
        except ValueError:
            return f"'{value}'는 유효하지 않은 정수입니다."
    
    elif data_type == "float":
        try:
            float(value)
            return f"'{value}'는 유효한 실수입니다."
        except ValueError:
            return f"'{value}'는 유효하지 않은 실수입니다."
    
    elif data_type == "email":
        pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        if re.match(pattern, value):
            return f"'{value}'는 유효한 이메일 주소입니다."
        else:
            return f"'{value}'는 유효하지 않은 이메일 주소입니다."

In [24]:
from langchain_openai import ChatOpenAI

# 도구 목록 생성
tools = [safe_divide, validate_input]

# 채팅 모델에 도구 바인딩
model = ChatOpenAI(model="gpt-4.1-mini")
model_with_tools = model.bind_tools(tools)

# 모델이 도구를 사용하도록 요청
response = model_with_tools.invoke("10을 3으로 나눠주세요")
print(response.tool_calls)

[{'name': 'safe_divide', 'args': {'a': 10, 'b': 3}, 'id': 'call_4H5UwdmpkAaOEMMuqLor0jC2', 'type': 'tool_call'}]


In [25]:
# 이메일 검증 요청
response = model_with_tools.invoke("test@example.com이 유효한 이메일인지 확인해주세요")
print(response.tool_calls)

[{'name': 'validate_input', 'args': {'value': 'test@example.com', 'data_type': 'email'}, 'id': 'call_dU50rZDSeBhDd8YiQVT9UkyI', 'type': 'tool_call'}]


#### 4.2 파일 처리 Tool


In [26]:
import json
import csv
from pathlib import Path

@tool
def read_file(file_path: str, encoding: str = "utf-8") -> str:
    """파일을 읽어서 내용을 반환합니다.
    
    Args:
        file_path: 읽을 파일 경로
        encoding: 파일 인코딩 (기본: utf-8)
    
    Returns:
        파일 내용 또는 오류 메시지
    """
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            content = file.read()
            return f"파일 '{file_path}' 내용:\n{content}"
    except FileNotFoundError:
        return f"오류: 파일 '{file_path}'를 찾을 수 없습니다."
    except Exception as e:
        return f"파일 읽기 오류: {str(e)}"

@tool
def write_file(file_path: str, content: str, encoding: str = "utf-8") -> str:
    """파일에 내용을 씁니다.
    
    Args:
        file_path: 쓸 파일 경로
        content: 쓸 내용
        encoding: 파일 인코딩 (기본: utf-8)
    
    Returns:
        작업 결과 메시지
    """
    try:
        with open(file_path, 'w', encoding=encoding) as file:
            file.write(content)
            return f"파일 '{file_path}'에 성공적으로 저장했습니다."
    except Exception as e:
        return f"파일 쓰기 오류: {str(e)}"

@tool
def process_json_data(json_string: str, operation: Literal["validate", "pretty", "minify"]) -> str:
    """JSON 데이터를 처리합니다.
    
    Args:
        json_string: JSON 문자열
        operation: 수행할 작업 ('validate', 'pretty', 'minify')
    
    Returns:
        처리 결과
    """
    try:
        data = json.loads(json_string)
        
        if operation == "validate":
            return "유효한 JSON 형식입니다."
        elif operation == "pretty":
            return json.dumps(data, indent=2, ensure_ascii=False)
        elif operation == "minify":
            return json.dumps(data, separators=(',', ':'), ensure_ascii=False)
        
    except json.JSONDecodeError as e:
        return f"JSON 파싱 오류: {str(e)}"
    except Exception as e:
        return f"처리 오류: {str(e)}"

In [27]:
from langchain_openai import ChatOpenAI

# 도구 목록 생성
tools = [read_file, write_file, process_json_data]

# 채팅 모델에 도구 바인딩
model = ChatOpenAI(model="gpt-4.1-mini")
model_with_tools = model.bind_tools(tools)

# 도구 딕셔너리 생성
tools_dict = {tool.name: tool for tool in tools}

# 모델이 도구를 사용하도록 요청
examples = [
    "test.txt 파일에 '안녕하세요, LangChain!'이라는 내용을 저장해주세요.",
    "test.txt 파일을 읽어주세요.",
    '{"name": "김철수", "age": 30, "city": "서울"} JSON을 예쁘게 포맷팅해주세요.',
    'config.json 파일에 {"database": {"host": "localhost", "port": 5432}} 내용을 저장하고 읽어주세요.'
]

for i, example in enumerate(examples, 1):
    print(f"예제 {i}: {example}")

    result = chat_with_tools(example, model_with_tools, tools_dict)
    print(f"결과: {result}")

    print("-" * 50)

예제 1: test.txt 파일에 '안녕하세요, LangChain!'이라는 내용을 저장해주세요.
결과: 파일 'test.txt'에 '안녕하세요, LangChain!'이라는 내용을 성공적으로 저장했습니다.
--------------------------------------------------
예제 2: test.txt 파일을 읽어주세요.
결과: test.txt 파일 내용을 읽었습니다. 파일 내용은 "안녕하세요, LangChain!" 입니다.
--------------------------------------------------
예제 3: {"name": "김철수", "age": 30, "city": "서울"} JSON을 예쁘게 포맷팅해주세요.
결과: {
  "name": "김철수",
  "age": 30,
  "city": "서울"
}
--------------------------------------------------
예제 4: config.json 파일에 {"database": {"host": "localhost", "port": 5432}} 내용을 저장하고 읽어주세요.
결과: config.json 파일에 {"database": {"host": "localhost", "port": 5432}} 내용을 성공적으로 저장했습니다.
파일 내용도 동일하게 잘 읽혔습니다. 다른 작업 도와드릴까요?
--------------------------------------------------


---

###  Tool Calling 사용 시 **고려사항**

- **모델 호환성**이 Tool Calling 성능에 직접 영향

- **명확한 도구 정의**가 모델의 이해도와 활용도 향상

- **단순한 기능**의 도구가 더 효과적으로 작동

- **과다한 도구**는 모델 성능 저하 유발

---

## Agent 개념과 실습

- **Agent**는 LLM을 의사결정 엔진으로 사용하여 복잡한 작업을 자동으로 수행하는 시스템
- Agent의 구성 요소:
    1. **LLM (추론 엔진)**: 상황을 분석하고 다음 행동을 결정
    2. **Tools (도구)**: Agent가 사용할 수 있는 기능들
    3. **Memory (메모리)**: 이전 대화나 작업 기록 저장
    4. **Prompt (프롬프트)**: Agent의 역할과 행동 지침 정의
- **LangGraph** 활용
    - **LangGraph**는 LangChain의 확장 도구로 **고급 에이전트 개발**을 지원
    - **그래프 기반 워크플로우**를 통해 복잡한 에이전트 로직을 구현할 수 있음 
    - 상태 관리와 **타입 안전성**을 통해 안정적인 에이전트 실행을 보장

In [30]:
from langgraph.prebuilt import create_react_agent
from typing import TypedDict, List
from langchain_core.messages import BaseMessage

# # 상태 정의
# class AgentState(TypedDict):
#     messages: List[BaseMessage]

# 기본 React Agent 생성
tools = [add, multiply, get_current_time, calculate_age, safe_divide, validate_input, read_file, write_file, process_json_data]
langgraph_agent = create_react_agent(llm, tools)

# 실행
response = langgraph_agent.invoke({
    "messages": [HumanMessage(content="오늘 42세인 사람의 생년월일은?")]
})

print("LangGraph Agent 응답:")
for message in response['messages']:
    if isinstance(message, AIMessage) and message.content:
        print(f"AI: {message.content}")

LangGraph Agent 응답:
AI: 오늘 42세인 사람의 생년도를 계산하려면 현재 연도에서 42를 빼면 됩니다. 현재 연도를 확인한 후 계산해드리겠습니다. 잠시만 기다려 주세요.
AI: 오늘이 2025년 8월 8일이고, 42세인 사람의 생년은 1983년입니다. 생년월일은 1983년 8월 8일 이전에 태어났음을 의미합니다. 정확한 생일은 추가 정보가 필요합니다.


In [31]:
response['messages']

[HumanMessage(content='오늘 42세인 사람의 생년월일은?', additional_kwargs={}, response_metadata={}, id='e8774344-fe2c-475a-87ca-046c88669318'),
 AIMessage(content='오늘 42세인 사람의 생년도를 계산하려면 현재 연도에서 42를 빼면 됩니다. 현재 연도를 확인한 후 계산해드리겠습니다. 잠시만 기다려 주세요.', additional_kwargs={'tool_calls': [{'id': 'call_zgbU43Tpc4dN7V6P1YoVTNOK', 'function': {'arguments': '{"format_type":"date"}', 'name': 'get_current_time'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 61, 'prompt_tokens': 602, 'total_tokens': 663, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_6f2eabb9a5', 'id': 'chatcmpl-C271MTxUPBKiT06aX3dnBoXbeb4cm', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d5147596-9f96-4ded-aff4-8dd0bc43829a-0', tool_ca